In [3]:
pip install ucimlrepo

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
import random
import math
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler

# ---------------------- MICROGRAD ENGINE ----------------------

class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out

    def __radd__(self, other): return self + other
    def __sub__(self, other): return self + (-other)
    def __rsub__(self, other): return Value(other) - self
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __rmul__(self, other): return self * other
    def __truediv__(self, other): return self * other**-1
    def __rtruediv__(self, other): return Value(other) / self

    def __neg__(self): return self * -1
    def __pow__(self, power):
        out = Value(self.data ** power, (self,), f'**{power}')

        def _backward():
            self.grad += power * self.data**(power - 1) * out.grad
        out._backward = _backward
        return out

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def exp(self):
        x = math.exp(self.data)
        out = Value(x, (self,), 'exp')

        def _backward():
            self.grad += x * out.grad
        out._backward = _backward
        return out

    def log(self):
        out = Value(math.log(self.data), (self,), 'log')

        def _backward():
            self.grad += (1 / self.data) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build(child)
                topo.append(v)
        build(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

# ---------------------- MICROGRAD NN ----------------------

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(0.0)

    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        return act.tanh()

    def parameters(self):
        return self.w + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]



In [28]:

# ---------------------- LOAD DATA ----------------------

auto_mpg = fetch_ucirepo(id=9)
X = auto_mpg.data.features
y = auto_mpg.data.targets
X = X.fillna(X.mean())  # fill NaN with column means


xs = X.values.tolist()
ys = y.values.flatten().tolist()

# Normalize features
scaler = StandardScaler()
xs = scaler.fit_transform(xs).tolist()



In [29]:
X

,displacement,cylinders,horsepower,weight,acceleration,model_year,origin
0,307.0,8,130.0,3504,12.0,70,1
1,350.0,8,165.0,3693,11.5,70,1
2,318.0,8,150.0,3436,11.0,70,1
3,304.0,8,150.0,3433,12.0,70,1
4,302.0,8,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...
393,140.0,4,86.0,2790,15.6,82,1
394,97.0,4,52.0,2130,24.6,82,2
395,135.0,4,84.0,2295,11.6,82,1
396,120.0,4,79.0,2625,18.6,82,1


In [31]:
(xs[0])

[1.0906036979541762,
 1.4981912580348042,
 0.6691960798919363,
 0.6308698741675453,
 -1.295498344294239,
 -1.627426292158321,
 -0.7151447804348392]

In [27]:
model = MLP(nin=len(xs[0]), nouts=[10, 1])  # input → 10 → 1

for epoch in range(100):
    total_loss = Value(0.0)
    for x_row, y_gt in zip(xs, ys):
        x_val = [Value(v) for v in x_row]
        y_pred = model(x_val)[0]
        loss = (y_pred - y_gt) ** 2
        total_loss += loss

    for p in model.parameters():
        p.grad = 0.0
    total_loss.backward()

    for p in model.parameters():
        p.data -= 0.001 * p.grad

    if epoch % 10 == 0:
        print(f"Epoch {epoch} | Loss: {total_loss.data:.4f}")


Epoch 0 | Loss: 240667.0670
Epoch 10 | Loss: 226001.2481
Epoch 20 | Loss: 226001.2324
Epoch 30 | Loss: 226001.2211
Epoch 40 | Loss: 226001.2125
Epoch 50 | Loss: 226001.2059
Epoch 60 | Loss: 226001.2006
Epoch 70 | Loss: 226001.1963
Epoch 80 | Loss: 226001.1927
Epoch 90 | Loss: 226001.1897


In [25]:
import math
for y in ys:
    assert not math.isnan(y), "Found NaN in targets"

for row in xs:
    assert all(not math.isnan(x) for x in row), "Found NaN in input"



In [ ]:
'''
origin	Country/Region of Manufacture
1	USA
2	Europe
3	Japan
'''

In [33]:
print("Sample input (scaled):", xs[0])
print("Target MPG:", ys[0])


Sample input (scaled): [1.0906036979541762, 1.4981912580348042, 0.6691960798919363, 0.6308698741675453, -1.295498344294239, -1.627426292158321, -0.7151447804348392]
Target MPG: 18.0


In [32]:
for k in range(20):

    #forward pass
    ypred = [model(x) for x in xs]
    loss = sum(((yout[0] - ygt)**2 for ygt, yout in zip(ys, ypred)))

    #backward pass
    for p in model.parameters():
        p.grad = 0.0
    loss.backward()

    #update
    lr = 0.001
    for p in model.parameters():
        p.data += -1 * lr * p.grad

    print(k, loss.data)

0 226001.1871495815
1 226001.18691577908
2 226001.18668547104
3 226001.18645858334
4 226001.18623504374
5 226001.18601478217
6 226001.1857977303


KeyboardInterrupt: 